Detección de caras con webcam

In [3]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [6]:

normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0


In [27]:
import cv2
import imutils #redimensionar
import numpy as np

cap = cv2.VideoCapture(0)
pumpkin = cv2.imread("pumpkin3.png",cv2.IMREAD_UNCHANGED)
#print('pumpkin.shape =',pumpkin.shape)
#cv2.imshow('pumpkin',pumpkin[:,:,3])
faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')


while True:
  ret, frame = cap.read()
  if ret == False:
    break
  
  faces = faceClassif.detectMultiScale(frame,1.3,5)
 
  for (x,y,w,h) in faces:
    #cv2.rectangle(frame, (x - 50, y - 25), (x + w+50, y + h+25), (0, 255, 0), 2)
      #Redimensionar la imagen 
    resized_image = imutils.resize(pumpkin,width=w,height=h)
    filas_img = resized_image.shape[0]
    col_img = w

    n_frame = frame[y: y + h, x: x + w]
    mask = resized_image[:,:,3]
    mask_inv=cv2.bitwise_not(mask)
    bg_black= cv2.bitwise_and(resized_image,resized_image,mask=mask)
    bg_black= bg_black[:,:,0:3]
    bg_frame = cv2.bitwise_and(n_frame,n_frame,mask=mask_inv)

    #Sumamos las dos imagenes

    final_pumpki = cv2.add(bg_black,bg_frame)
    frame[y: y + h, x: x + w] = final_pumpki
    
    #Aplicamos filtros al frame
  
    cv2.imshow('Halloween',frame)
    key = cv2.waitKey(1) & 0xFF

  if key == 27:
      break


# Liberamos los recursos
cap.release()
cv2.destroyAllWindows()


Camera 0


ValueError: could not broadcast input array from shape (219,219,4) into shape (0,219,3)